In [1]:
import pandas as pd
import numpy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

from sklearn.decomposition import PCA
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

import time
from datetime import datetime 

In [2]:
test = pd.read_csv("C:\\MyFoulder\\SCE\שנה ג\\סמסטר ב\\מבוא ללמידה עמוקה\\2\\test_ex2_dl2021b.csv")
train = pd.read_csv("C:\\MyFoulder\\SCE\שנה ג\\סמסטר ב\\מבוא ללמידה עמוקה\\2\\train_ex2_dl2021b.csv")
train

,sid,sentence,label
0,0,Government also performs the important job of...,1
1,1,The second feature is the existence of sharp p...,1
2,2,Allen Schick .\n,0
3,3,"Yet , not all bureaucracies are alike .\n",0
4,4,Approximately 75 percent of recently emerging ...,1
...,...,...,...
1204,1204,"For example , a municipal bond is issued by ci...",1
1205,1205,The line - item veto gives governors the abili...,0
1206,1206,"When reading Freud ’s theories , it is importa...",0
1207,1207,"Consequently , all of these organisms also hav...",0


In [3]:
test

,sid,sentence
0,0,"The equal - time rule , for instance , states ..."
1,1,"Rather , obsessions are characterized as pers..."
2,2,Choices made along the labor - leisure budget ...
3,3,"Social and religious leaders — rabbis , priest..."
4,4,The processes of homeostasis and temperature c...
...,...,...
425,425,The rays themselves are called nuclear radiati...
426,426,"According to Malthusian theory , three factors..."
427,427,At the highest level of organization ( [ link ...
428,428,This is analogous to a phase transition in the...


In [4]:
train['sentence'] = train['sentence'].str.replace('[^\w\s]','')
train['sentence'] = train['sentence'].str.replace('[^a-zA-z ]','')

test['sentence'] = test['sentence'].str.replace('[^\w\s]','')
test['sentence'] = test['sentence'].str.replace('[^a-zA-z ]','')

In [5]:
train.isnull().sum()

sid         0
sentence    0
label       0
dtype: int64

In [6]:
test.isnull().sum()

sid         0
sentence    0
dtype: int64

In [7]:
X = train.drop(['sid', 'label'], axis=1)
X

,sentence
0,Government also performs the important job of...
1,The second feature is the existence of sharp p...
2,Allen Schick
3,Yet not all bureaucracies are alike
4,Approximately percent of recently emerging in...
...,...
1204,For example a municipal bond is issued by cit...
1205,The line item veto gives governors the abilit...
1206,When reading Freud s theories it is important...
1207,Consequently all of these organisms also have...


In [8]:
y = train['label']
y

0       1
1       1
2       0
3       0
4       1
       ..
1204    1
1205    0
1206    0
1207    0
1208    1
Name: label, Length: 1209, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
X_train
#X_test
#y_train
#y_test

,sentence
253,Indeed voting itself may reflect merely a la...
429,Think about what you do when you walk into an ...
734,The two most important laws are the Freedom of...
1116,In under the cloak of anonymity the associa...
282,Often the process begins with a single motive...
...,...
581,Auditory hallucinations hearing voices occur...
688,These germ layers are programmed to develop in...
265,The dorsal cavity contains the cranial and the...
1127,Consider this example You live in Washington ...


In [11]:
sentences = [w.split() for w in X_train['sentence']]
print(len(sentences))
model = Word2Vec(sentences, min_count=1, vector_size=300)
print(model)

967
Word2Vec(vocab=5214, vector_size=300, alpha=0.025)


In [12]:
words = list(model.wv.index_to_key)
#print(model.wv.get_vector("the", norm=True))
#model.wv.most_similar('data')

In [3]:
sen = list(X_train['sentence'])
avgs = []
vecs = []
for s in sen:
    for w in s.split():
        #if w in words:
        vecs.append(numpy.array(model.wv.get_vector(w, norm=True)))
    if len(s.split()) == 0:
        #avgs.append(sum(vecs)/1)
        avgs.append(numpy.array([avgs[0][0] for i in range(300)]))
        #print(avgs[len(avgs)-1])
    else:
        avgs.append(sum(vecs)/len(s.split()))
    vecs.clear()


NameError: name 'X_train' is not defined

In [14]:
start_time = datetime.now() 

In [15]:
def create_baseline():
    model = Sequential()
    model.add(Dense(100, input_dim=300, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=10, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, numpy.array(avgs), y_train, cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 68.05% (0.27%)


In [16]:
print('Time elapsed in (hh:mm:ss.ms): "{}"'.format(datetime.now() - start_time))

Time elapsed in (hh:mm:ss.ms): "0:00:07.486930"


In [17]:
model2 = create_baseline()
model2.fit(numpy.array(avgs), y_train.values, epochs=100, batch_size=10, shuffle=True)

Epoch 1/10
97/97 [==============================] - 0s 457us/step - loss: 0.6518 - accuracy: 0.6632
Epoch 2/10
97/97 [==============================] - 0s 468us/step - loss: 0.6521 - accuracy: 0.6581
Epoch 3/10
97/97 [==============================] - 0s 468us/step - loss: 0.6170 - accuracy: 0.6972
Epoch 4/10
97/97 [==============================] - 0s 468us/step - loss: 0.6146 - accuracy: 0.7018
Epoch 5/10
97/97 [==============================] - 0s 467us/step - loss: 0.6249 - accuracy: 0.6874
Epoch 6/10
97/97 [==============================] - 0s 467us/step - loss: 0.6252 - accuracy: 0.6901
Epoch 7/10
97/97 [==============================] - 0s 467us/step - loss: 0.6239 - accuracy: 0.6840
Epoch 8/10
97/97 [==============================] - 0s 468us/step - loss: 0.6287 - accuracy: 0.6813
Epoch 9/10
97/97 [==============================] - 0s 478us/step - loss: 0.6096 - accuracy: 0.7071
Epoch 10/10
97/97 [==============================] - 0s 467us/step - loss: 0.6329 - accuracy: 0.6669

In [18]:
sentences = [w.split() for w in test['sentence']]
print(len(sentences))
model = Word2Vec(sentences, min_count=1, vector_size=300)
print(model)

430
Word2Vec(vocab=3133, vector_size=300, alpha=0.025)


In [19]:
sen = list(test['sentence'])
avgs = []
vecs = []
for s in sen:
    for w in s.split():
        vecs.append(numpy.array(model.wv.get_vector(w, norm=True)))
    if len(s.split()) == 0:
        avgs.append(numpy.array([avgs[0][0] for i in range(300)]))
    else:
        avgs.append(sum(vecs)/len(s.split()))
    vecs.clear()


In [20]:
test_pred = (model2.predict_classes(numpy.array(avgs)) > 0.5).astype("int32")
#test_pred = model2.predict_classes(numpy.array(avgs))

C:\Users\jenga\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [21]:
#test_pred

In [22]:
#len(test_pred)

In [23]:
j = 0
for i in range(430):
    if test_pred[i]==1:
        j+=1
print(j)

0


In [24]:
# df3 = pd.read_csv("C:\\MyFoulder\\SCE\שנה ג\\סמסטר ב\\מבוא ללמידה עמוקה\\2\\sample_submission_ex2_dl2021b.csv")
# df3['label'] = test_pred

# df3.to_csv('Project2.15.csv', index = False)

#2.15 30

In [25]:
k = 0
for i in range(430):
    if test_pred[i]==1:
        k+=1
        print(i)
print('\nlen: ', k)


len:  0


In [26]:
# kuku = pd.read_csv("Project2.5.csv")
# kuku_len = kuku.shape
# kuku_len

# 74
# 116
# 141
# 159
# 170
# 219
# 304
# 316
# 325
# 343
# 399
# 411
# 413
# 416
# 425

# len:  15

# 34
# 93
# 184
# 247
# 322
# 329
# 330
# 346
# 363
# 368
# 410

# len:  11

# 20
# 56
# 78
# 338
# 358
# 383
# 399
# 406

# len:  8

# # 2
# # 193
# # 244
# # 259
# # 318
# # 328
# # 338
# # 388
# # 402

# # len:  9


# 12
# 39
# 115
# 326
# 399
# 401
# 425

# len:  7

In [27]:
# print('X_train len: ',len(X_train))
# print('y_train len: ',len(y_train))
# print('avgs len: ',len(avgs))
# print('vecs len: ',len(vecs))
# print('words: ',len(words))
# print('X_train: ',X_train.shape)
# print('y_train: ',y_train.shape)

In [28]:
# for i in range(len(avgs)):
#     if isinstance(avgs[i], float):
#         print('fdbsfbsd')
#         print(avgs[i],i)
#         avgs[i]=numpy.array(avgs[i-1])

In [29]:
# df = pd.read_csv("C:\MyFoulder\SCE\שנה ג\סמסטר ב\מבוא ללמידה עמוקה\kaggle\\train.csv")
# df
# df.isnull().sum()
# X = df.drop(['id', 'label'], axis=1)
# X
# y = df['label']
# y
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# X_train
# #X_test
# #y_train
# #y_test
# y_train.shape
# sentences = [w.split() for w in X_train['data']]
# print(len(sentences))
# model = Word2Vec(sentences, min_count=1, vector_size=300)
# print(model)
# words = list(model.wv.index_to_key)
# words
# #print(model.wv.get_vector("tea", norm=True))
# #model.wv.most_similar('data')
# q = []
# for x in words:
#     q.append(model.wv.get_vector(x, norm=True).tolist())
# print(type(q[0]))

In [30]:
# from numpy import loadtxt
# from keras.models import Sequential
# from keras.layers import Dense


# # define the keras model
# model = Sequential()
# model.add(Dense(300, input_dim=300, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# # compile the keras model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# # fit the keras model on the dataset
# model.fit(avgs, y, epochs=150, batch_size=10)
# # evaluate the keras model
# _, accuracy = model.evaluate(numpy.asarray(avgs), y_train)
# print('Accuracy: %.2f' % (accuracy*100))